In [1]:
from nltk import ngrams
import pandas as pd
import numpy as np 
from pandas import Series
import pickle
import cPickle
import os
pd.options.display.max_rows=3000
root_dir = os.path.abspath('../..')
task_dir= os.path.join(root_dir,'user_profile')
data_dir=os.path.join(task_dir,'data')
final_data=os.path.join(data_dir,"final_scrap")
from sklearn import svm
from sklearn.utils import shuffle
from scipy.sparse import *
from sklearn.metrics import precision_recall_fscore_support

In [2]:
def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((  loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])

In [12]:
#vectorizer is a feature generator for special characters 
special_characters=['#','/', '%', '~', '=','?','&',"-","(",")","+","@","[","]","'","*"]
with open('feature_genrator_special_charac.pkl', 'rb') as fid:
    vectorizer_spcl=cPickle.load(fid)

In [13]:
"""Opens and stores the vocab for generating 3-gram features into vocab_ list for vectorizer input"""

the_filename="test_vocab"
with open(the_filename, 'rb') as f:
    vocab_ = pickle.load(f)

from sklearn.feature_extraction.text import CountVectorizer
vectorizerY= CountVectorizer(min_df=1)
vectorizerY.fit_transform(vocab_)
"""generating feature extraction for vocab"""

'generating feature extraction for vocab'

In [14]:
import re
import urlparse
def hostname(x):
    return urlparse.urlparse(x).hostname

def path(x):
    return urlparse.urlparse(x).path

def preprocessor(list_):
    return list("".join(list_))

def gram_generator(x):
    grams = ngrams(x, 3)
    return ["".join(gram) for gram in grams]

def character_extracter(x):
    return x.str.lower().str.replace("http",".").str.replace("www",".").str.findall(r"\w+").apply(preprocessor)


def joined(list_):
    return " ".join(list_)

In [15]:
def encoder(x):
    x=x.str.replace("http://|https://","").str.findall("[^\:\w\.]").apply(joined)
    for n,spcl in enumerate(special_characters):
        x=x.str.replace(spcl,str(n+10))
    return x

In [16]:
def complete_processor(x,type_=1):
    x=x.iloc[np.random.permutation(x.shape[0])]
    x=character_extracter(x)
    x=x.apply(gram_generator).apply(joined).values
    x_=vectorizerY.transform(x)
    if type_==1:
        y_=csr_matrix(np.ones((x_.shape[0],1)))
    elif type_==0:
        y_=csr_matrix(np.zeros((x.shape[0],1)))

    return x_,y_

In [17]:
def len_converter(x):
    return csc_matrix(np.reshape(x.str.len().as_matrix(),(x.shape[0],1)))

In [18]:
def label_formatter(y):
    return np.reshape(y.toarray(),(y.shape[0],))

In [19]:
#load the saved files
profile_urls=pd.read_csv(os.path.join(final_data,"profile_urls_from_working_data.csv"),header=None)[1]
non_profile_urls=pd.read_csv(os.path.join(final_data,"non_profile_urls_from_working_data.csv"),header=None)[1]

x_positive_len=len_converter(profile_urls)
x_negative_len=len_converter(non_profile_urls)

x_spcl_positive=vectorizer_spcl.transform(encoder(profile_urls).values)
x_spcl_negative=vectorizer_spcl.transform(encoder(non_profile_urls).values)

x_positive,y_positive=complete_processor(profile_urls,type_=1)
x_negative,y_negative=complete_processor(non_profile_urls,type_=0)

x_pos=hstack([x_positive,x_spcl_positive,x_positive_len])
x_neg=hstack([x_negative,x_spcl_negative,x_negative_len])

In [24]:
np.random.seed(0) #to recreate results
x_data=vstack([x_neg,x_pos])
y_data=vstack([y_negative,y_positive])
x_data,y_data=shuffle(x_data,y_data)

In [25]:
x_train=x_data[:60000,:]
y_train=y_data[:60000,:]
x_val=x_data[60000:75000,:]
y_val=y_data[60000:75000,:]
x_test=x_data[75000:,:]
y_test=y_data[75000:,:]

In [26]:
clf=svm.LinearSVC(C=0.90).fit(x_train,label_formatter(y_train))

In [27]:
precision_recall_fscore_support(label_formatter(y_test),clf.predict(x_test))

(array([ 0.98606403,  0.93702866]),
 array([ 0.98740288,  0.93071161]),
 array([ 0.986733  ,  0.93385945]),
 array([13257,  2670]))

In [28]:
precision_recall_fscore_support(label_formatter(y_val),clf.predict(x_val))

(array([ 0.98734278,  0.94052502]),
 array([ 0.98844437,  0.93515498]),
 array([ 0.98789327,  0.93783231]),
 array([12548,  2452]))

In [29]:
precision_recall_fscore_support(label_formatter(y_train),clf.predict(x_train))

(array([ 0.9952434 ,  0.98421161]),
 array([ 0.99692989,  0.97570891]),
 array([ 0.99608593,  0.97994182]),
 array([50161,  9839]))

In [30]:
url  = "www.facebook.com/abhiagwl"

In [31]:
def url_to_input(url,model):
    x = hstack([complete_processor(Series(url))[0],vectorizer_spcl.transform(encoder(Series(url)).values),len_converter(Series(url))])
    output = model.predict(x)
    if (output[0]==1):
        print "Profile"
    else :
        print "Not Profile"

In [32]:
%time url_to_input(url,clf)

Profile
CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 10.7 ms
